In [ ]:
import model_based_mvpa as momvpa
from model_based_mvpa.preprocessing.bids import *
from model_based_mvpa.preprocessing.events import *
from model_based_mvpa.data.loader import *
from model_based_mvpa.models.model import *

In [ ]:
root = '/data2/project_model_based_fmri/ds000005/'

In [ ]:
layout = BIDSLayout(root, derivatives=True)

In [ ]:
X, masked_data, layout = bids_preprocess(root, smoothing_fwhm=None, zoom=(2, 2, 2), ncore=4, nthread=4)

In [ ]:
def example_tom_adjust_columns(row):
    ## rename data in a row to the name which can match hbayesdm.ra_prospect requirements ##
    row["gamble"] = 1 if row["respcat"] == 1 else 0
    row["cert"] = 0
    return row

In [ ]:
def example_tom_condition(row):
    # include all trial data
    return True

In [ ]:
def example_tom_modulation(row, param_dict):
    ## calculate subjectives utility for choosing Gamble over Safe option
    ## prospect theory with loss aversion and risk aversion is adopted
    
    modulation = (row["gain"] ** param_dict["rho"]) - (param_dict["lambda"] * (row["loss"] ** param_dict["rho"]))
    row["modulation"] = modulation
    return row

In [ ]:
dm_model, df_events, sig  nals, time_masks, _ = \
    events_preprocess(root, layout=layout,
                      preprocess=example_tom_adjust_columns,
                      condition=example_tom_condition,
                      modulation=example_tom_modulation,
                      dm_model='ra_prospect')

## model

In [ ]:
from model_based_mvpa.models.regressor import *
from model_based_mvpa.utils.coef2map import *
import time

In [ ]:
X, y = prepare_dataset(root=Path(layout.derivatives["fMRIPrep"].root))

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = mlp_regression_v2(X, y,
                       layer_dims=[1024, 1024],
                       activation_func='linear',
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=3,
                       verbose=1)

task_name = 'tom2007_mlp_v2'

result = get_map(coefs, masked_data, task_name, map_type='z', save_path=None, smoothing_sigma=1)

print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = penalized_linear_regression(X, y,
                             alpha=0.001,
                             lambda_par=2.0,
                             epochs=100,
                             patience=30,
                             batch_size=256,
                             N=30,
                             verbose=1)

task_name = 'tom2007_penalized_linear'

result = get_map(coefs, masked_data, task_name, map_type='z', save_path=None, smoothing_sigma=1)

print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = penalized_linear_regression(X, y,
                             alpha=0.001,
                             lambda_par=2.0,
                             epochs=100,
                             patience=30,
                             batch_size=256,
                             N=30,
                             verbose=1)

task_name = 'tom2007_penalized_linear'

result = get_map(coefs, masked_data, task_name, map_type='z', save_path=None, smoothing_sigma=1)

print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = elasticnet(X, y,max_use_sample_N=10000,
             alpha=0.001,
             n_jobs=16,
             N=3,
             verbose=3)

task_name = 'tom2007_elasticnet'

result = get_map(coefs, masked_data, task_name, map_type='z', save_path=None, smoothing_sigma=1)

print(time.strftime('%c', time.localtime(time.time())))